$\newcommand{\xv}{\mathbf{x}}
\newcommand{\Xv}{\mathbf{X}}
\newcommand{\yv}{\mathbf{y}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\av}{\mathbf{a}}
\newcommand{\Wv}{\mathbf{W}}
\newcommand{\wv}{\mathbf{w}}
\newcommand{\tv}{\mathbf{t}}
\newcommand{\Tv}{\mathbf{T}}
\newcommand{\muv}{\boldsymbol{\mu}}
\newcommand{\sigmav}{\boldsymbol{\sigma}}
\newcommand{\phiv}{\boldsymbol{\phi}}
\newcommand{\Phiv}{\boldsymbol{\Phi}}
\newcommand{\Sigmav}{\boldsymbol{\Sigma}}
\newcommand{\Lambdav}{\boldsymbol{\Lambda}}
\newcommand{\half}{\frac{1}{2}}
\newcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\newcommand{\argmin}[1]{\underset{#1}{\operatorname{argmin}}}$

# Assignment 6: Min-Conflicts

*Ben Newell*

## Given Functions

These functions were given and described in class. They are included here to for the implemented code to function properly. 

In [2]:
import random

def min_conflicts(vars, domains, constraints, neighbors, max_steps=1000): 
    """Solve a CSP by stochastic hillclimbing on the number of conflicts."""
    # Generate a complete assignment for all vars (probably with conflicts)
    current = {}
    for var in vars:
        val = min_conflicts_value(var, current, domains, constraints, neighbors)
        current[var] = val
    # Now repeatedly choose a random conflicted variable and change it
    for i in range(max_steps):
        conflicted = conflicted_vars(current,vars,constraints,neighbors)
        if not conflicted:
            return (current,i)
        var = random.choice(conflicted)
        val = min_conflicts_value(var, current, domains, constraints, neighbors)
        current[var] = val
    return (None,None)

In [3]:
def min_conflicts_value(var, current, domains, constraints, neighbors):
    """Return the value that will give var the least number of conflicts.
    If there is a tie, choose at random."""
    return argmin_random_tie(domains[var],
                             lambda val: nconflicts(var, val, current, constraints, neighbors)) 

In [4]:
def conflicted_vars(current,vars,constraints,neighbors):
    "Return a list of variables in current assignment that are in conflict"
    return [var for var in vars
            if nconflicts(var, current[var], current, constraints, neighbors) > 0]

In [5]:
def nconflicts(var, val, assignment, constraints, neighbors):
    "Return the number of conflicts var=val has with other variables."
    # Subclasses may implement this more efficiently
    def conflict(var2):
        val2 = assignment.get(var2, None)
        return val2 != None and not constraints(var, val, var2, val2)
    return len(list(filter(conflict, neighbors[var])))

In [6]:
def argmin_random_tie(seq, fn):
    """Return an element with lowest fn(seq[i]) score; break ties at random.
    Thus, for all s,f: argmin_random_tie(s, f) in argmin_list(s, f)"""
    best_score = fn(seq[0]); n = 0
    for x in seq:
        x_score = fn(x)
        if x_score < best_score:
            best, best_score = x, x_score; n = 1
        elif x_score == best_score:
            n += 1
            if random.randrange(n) == 0:
                    best = x
    return best

## Implemented Functions

`build_tuples` is a helper function for `schedule`. It takes a list of times and a list of rooms and builds a list containing each combination of times and rooms. 

In [7]:
def build_tuples(times, rooms):
    retlist = []
    for t in times:
        for r in rooms:
            retlist.append((r,t))
    return retlist

First, `schedule` builds the list of domains. This is simply all the possible combinations of times and rooms that is built by `build_tuples`. Then it creates the domains dictionary by giving this list to each class as a key to a dictionary. The neighbors dictionary is then created where each class is a key that has every other class as a neighbor in its value list. Finally, `min_conflicts` is called which returns a non-conflicting solution if one is found.

In [8]:
def schedule(classes, times, rooms, max_steps):
    #classes are your variables
    # times and rooms are your domains, these are the things constraints ok checkts
    #max_steps is the time to run the min_constraints.
    
    #build full list of tuples.
    domainList = build_tuples(times, rooms)

    # create domains. To start, each class has each room and time
    domains = {key: domainList for key in classes} 
    
    #create neighbors
    neighbors = {key: [c for c in classes if c != key] for key in classes}
    
    solution, steps = min_conflicts(classes, domains, constraints_ok, neighbors, max_steps)
    
    return solution, steps

`constraints_ok` takes a two class value pairs and compares them to see if they have any scheduling conflicts. First it checks to see whether the classes occur at the same time. If they don't then there cannot be a conflict, so True is returned. Then it checks two conditions at once. One is valid for all classes, if they occur at the same time and the same room, then there is a conflict so the function returns false. In addition, it checks to see whether the third digit, which indicates the class level, is the same. If it is, then the the constraints are broken so False is returned. If both of these constraints are passed, the we return True.

In [9]:
def constraints_ok(class_name_1, value_1, class_name_2, value_2):
    ##class_name_1&2 are just string names.
    ##value_1&2 are tuples of (class name, time). Both strings
    class1_room, class1_time = value_1
    class2_room, class2_time = value_2
    
    ##should we check to see if names are the same? that doesn't violate constraints.
    ##Can't be in the same room at the same time. True for everyone.
    if (class1_time != class2_time):
        #we're good no matter what in this case.
        return True
    ##classes with the same first digit cannot be at the same time. 
    elif (class1_room != class2_room and class_name_1[2] != class_name_2[2]):
        #in this case their in different rooms, so same time is ok unless they have same level
        return True
    
    
    return False

The `display` function takes the assignments result from `schedule` along with a list of room and times and displays them in a reasonable manner. It uses format strings to adapt to the number of rooms and times passed in and display them in a reasonable way. 

In [137]:
def display(assignments, rooms, times):
    length = len(rooms)
    print('   ',('   {} '*length).format(*rooms))
    print('---' + '-----------' * length)
    for t in times:
        classes = [class_number for class_number,pair in assignments.items() if pair[1] == t]
        print(t + ('   {}   '*len(classes)).format(*classes))
        #print(classes)

### Extra Credit Functions

This will prefer a solution that has later meeting times and schedules the entry level classes around one and two.


`schedule_advanced` works by calling schedule for `max_tries` and keeping track of whichever trial has the lowest number of the violated preferences (specific meeting times). 

In [117]:
def schedule_advanced(classes, times, rooms, max_steps, max_tries):
    # call schedule and get the returns
    best_schedule, best_steps = schedule(classes, times, rooms, max_steps)
    min_count = preference_count(best_schedule)
    # min solution is set to that one.
    # min bad things is set to bad_things(schedule)
    # for number of steps
    for _ in range(max_tries):
        new_schedule, new_steps = schedule(classes, times, rooms, max_steps)
        new_count = preference_count(new_schedule)
        if new_count < min_count:
            best_schedule = new_schedule
            min_count = new_count
            best_steps = new_steps
    return best_schedule, best_steps
    # if min bad things > badThings[step], replace

`preference_count` returns the number of special preferences violated by a given schedule. It goes through each class's meeting time and checks to see whether it occurs early or late or a lunch because people prefere not to have classes at these times. Then it has a special preference for scheduling the intro level classes during the middle of the day so it is easier for students to fit them in. After counting all of these, it returns the count. 

In [127]:
def preference_count(schedule):
    #returns a count of violated preference.
    count = 0
    # count each class that occurs at 9, 12 or 4.
    for class_name in schedule:
        _, time = schedule[class_name]

        if time == " 9 am" or time == "12 pm" or time == " 4 pm":
            count += 1
        if class_name == "CS163" or class_name == "CS164":
            if not (time == " 1 pm" or time == " 2 pm"):
                count += 1
    return count
        
    # do not count if CS163 and CS164 meeting at 1 pm or 2 pm, otherwise count for each non occurance.

## Testing 

Testing `constraints_ok`

In [141]:
assert(not constraints_ok("CS160", ("CLARK 101", '9 am'), "CS200", ("CLARK 101", '9 am')))
assert(not constraints_ok("CS160", ("CLARK 101", '9 am'), "CS161", ("CSB130", '9 am')))
assert(constraints_ok("CS160", ("CLARK 101", '9 am'), "CS161", ("CSB130", '8 am')))
assert(constraints_ok("CS160", ("CLARK 101", '9 am'), "CS270", ("CSB130", '9 am')))
print("constraints_ok passed all tests!")

constraints_ok passed all tests!


Testing `build_tuples`

In [142]:
domainList = build_tuples(times, rooms)
domains = {key: domainList for key in classes}
domains
#each class should have all combos as its list

{'CS160': [('CSB 130', ' 9 am'),
  ('CSB 325', ' 9 am'),
  ('CSB 425', ' 9 am'),
  ('CSB 130', '10 am'),
  ('CSB 325', '10 am'),
  ('CSB 425', '10 am'),
  ('CSB 130', '11 am'),
  ('CSB 325', '11 am'),
  ('CSB 425', '11 am'),
  ('CSB 130', '12 pm'),
  ('CSB 325', '12 pm'),
  ('CSB 425', '12 pm'),
  ('CSB 130', ' 1 pm'),
  ('CSB 325', ' 1 pm'),
  ('CSB 425', ' 1 pm'),
  ('CSB 130', ' 2 pm'),
  ('CSB 325', ' 2 pm'),
  ('CSB 425', ' 2 pm'),
  ('CSB 130', ' 3 pm'),
  ('CSB 325', ' 3 pm'),
  ('CSB 425', ' 3 pm'),
  ('CSB 130', ' 4 pm'),
  ('CSB 325', ' 4 pm'),
  ('CSB 425', ' 4 pm')],
 'CS163': [('CSB 130', ' 9 am'),
  ('CSB 325', ' 9 am'),
  ('CSB 425', ' 9 am'),
  ('CSB 130', '10 am'),
  ('CSB 325', '10 am'),
  ('CSB 425', '10 am'),
  ('CSB 130', '11 am'),
  ('CSB 325', '11 am'),
  ('CSB 425', '11 am'),
  ('CSB 130', '12 pm'),
  ('CSB 325', '12 pm'),
  ('CSB 425', '12 pm'),
  ('CSB 130', ' 1 pm'),
  ('CSB 325', ' 1 pm'),
  ('CSB 425', ' 1 pm'),
  ('CSB 130', ' 2 pm'),
  ('CSB 325', ' 2 pm'

Looks like the large list of domains is being built correctly.

Testing `schedule` with the classes, times and rooms required by the assignment. 

In [157]:
classes = ['CS160', 'CS163', 'CS164',
           'CS220', 'CS270', 'CS253',
           'CS320', 'CS314', 'CS356', 'CS370',
           'CS410', 'CS414', 'CS420', 'CS430', 'CS440', 'CS445', 'CS453', 'CS464',
           'CS510', 'CS514', 'CS535', 'CS540', 'CS545']

times = [' 9 am',
         '10 am',
         '11 am',
         '12 pm',
         ' 1 pm',
         ' 2 pm',
         ' 3 pm',
         ' 4 pm']

rooms = ['CSB 130', 'CSB 325', 'CSB 425']

In [158]:
max_steps = 100
assignments, steps = schedule(classes, times, rooms, max_steps)
print('Took', steps, 'steps')
print(assignments)

Took 0 steps
{'CS160': ('CSB 425', ' 2 pm'), 'CS163': ('CSB 130', '12 pm'), 'CS164': ('CSB 130', '10 am'), 'CS220': ('CSB 130', ' 4 pm'), 'CS270': ('CSB 425', ' 9 am'), 'CS253': ('CSB 130', ' 2 pm'), 'CS320': ('CSB 425', '12 pm'), 'CS314': ('CSB 130', '11 am'), 'CS356': ('CSB 425', ' 3 pm'), 'CS370': ('CSB 425', '10 am'), 'CS410': ('CSB 325', ' 1 pm'), 'CS414': ('CSB 325', ' 2 pm'), 'CS420': ('CSB 130', ' 3 pm'), 'CS430': ('CSB 130', ' 9 am'), 'CS440': ('CSB 325', '10 am'), 'CS445': ('CSB 325', '11 am'), 'CS453': ('CSB 325', '12 pm'), 'CS464': ('CSB 325', ' 4 pm'), 'CS510': ('CSB 130', ' 1 pm'), 'CS514': ('CSB 325', ' 3 pm'), 'CS535': ('CSB 425', ' 4 pm'), 'CS540': ('CSB 325', ' 9 am'), 'CS545': ('CSB 425', '11 am')}


Its hard to see whether `schedule` is working correctly at this point without the nice formatting from `display`, but it looks like the correctly length and output is being generated. 

Testing `display` on the output from above.

In [159]:
display(assignments, rooms, times)

       CSB 130    CSB 325    CSB 425 
------------------------------------
 9 am   CS270      CS430      CS540   
10 am   CS164      CS370      CS440   
11 am   CS314      CS445      CS545   
12 pm   CS163      CS320      CS453   
 1 pm   CS410      CS510   
 2 pm   CS160      CS253      CS414   
 3 pm   CS356      CS420      CS514   
 4 pm   CS220      CS464      CS535   


With this readable format we can see that `schedule` finds a working schedule using `min_conflicts` that does not violate the constraints function.

Using larger sets to test whether `display` handles them correctly.

In [163]:
classes_ex = ['CS100', 'CS160', 'CS163', 'CS164',
           'CS220', 'CS270', 'CS253',
           'CS320', 'CS314', 'CS356', 'CS370',
           'CS410', 'CS414', 'CS420', 'CS430', 'CS440', 'CS445', 'CS453', 'CS464',
           'CS510', 'CS514', 'CS535', 'CS540', 'CS545']

times_ex = [' 8 am',
         ' 9 am',
         '10 am',
         '11 am',
         '12 pm',
         ' 1 pm',
         ' 2 pm',
         ' 3 pm',
         ' 4 pm']

rooms_ex = ['CSB 130', 'CSB 325', 'CSB 425', 'CLARK 101']

In [164]:
assignments, steps = schedule(classes_ex, times_ex, rooms_ex, max_steps)
display(assignments, rooms_ex, times_ex)

       CSB 130    CSB 325    CSB 425    CLARK 101 
-----------------------------------------------
 8 am   CS160      CS270      CS414   
 9 am   CS370      CS410      CS545   
10 am   CS440      CS535   
11 am   CS464      CS510   
12 pm   CS100      CS253      CS314      CS420   
 1 pm   CS163      CS220      CS320      CS453   
 2 pm   CS445      CS540   
 3 pm   CS356   
 4 pm   CS164      CS430      CS514   


Looks like it expands the columns fairly well. 

Testing `schedule_advanced` and `preference_count` with a reduced class list to see whether it adjusts for preferences. 

In [165]:
# shortened set to see what solutions it comes up with
classes = ['CS160', 'CS163', 'CS164',
           'CS220', 'CS270', 'CS253',
           'CS320', 'CS314', 'CS356', 'CS370',
           'CS410', 'CS414', 'CS420']
times = [' 9 am',
         '10 am',
         '11 am',
         '12 pm',
         ' 1 pm',
         ' 2 pm',
         ' 3 pm',
         ' 4 pm']
rooms = ['CSB 130', 'CSB 325', 'CSB 425']

In [166]:
best, steps = schedule_advanced(classes, times, rooms, 100, 5000)
display(best, rooms, times)

       CSB 130    CSB 325    CSB 425 
------------------------------------
 9 am   CS270   
10 am   CS220      CS356   
11 am   CS320   
12 pm   CS314   
 1 pm   CS164      CS370      CS420   
 2 pm   CS163      CS414   
 3 pm   CS160      CS253      CS410   
 4 pm


In [167]:
preference_count(best)

2

In [168]:
best, steps = schedule_advanced(classes_ex, times_ex, rooms_ex, 100, 5000)
display(best, rooms_ex, times_ex)

       CSB 130    CSB 325    CSB 425    CLARK 101 
-----------------------------------------------
 8 am   CS160      CS370      CS430   
 9 am   CS356      CS464   
10 am   CS270      CS445      CS545   
11 am   CS100      CS453      CS510   
12 pm   CS410      CS535   
 1 pm   CS164      CS220      CS414   
 2 pm   CS163      CS440      CS514   
 3 pm   CS253      CS320      CS420      CS540   
 4 pm   CS314   


In [169]:
preference_count(best)

5

## Conclusion and Notes

Ovearall, `min_conflicts` does a good job of finding a solution without conflicts. When I tried it, the steps to solution was very variable, sometimes landing on a solution right away and at others taking 18 or so steps to get there. I have noticed that it tends to favor the first class room very heavily. The expanded sets show this problem fairly well. It stacks everything into CSB 130 even when there is plenty of space in CLARK. Possibly randomization in the building of domains could help to spread out the assignments some. This becomes a problem with the extra credit because even though there is space to move around the assignments and violate fewer preferences, min_conflicts doesn't tend to return those sorts of solutions